In [1]:
import re
import sys
from nltk.corpus import stopwords
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Using TensorFlow backend.


In [2]:
data=pd.read_csv('Data.csv')

In [3]:
data.head()

,id,title,motion,manual_motion,govt_opp_motion,motion_party_affiliation,utt1,utt2,utt3,utt4,utt5,manual_speech,vote_speech,party_affiliation,name,rebellion
0,1,Doctor Assisted Dying,That leave be given to bring in a Bill to enab...,1,1,Lab,I trust that the House will not feel that ther...,If one considers the argument from a religious...,I accept that we shall all face death at one t...,"When we listen to my hon. Friend's case, we mu...",I do not think that we want that sort of socie...,0,0,Lab,Kevin McNamara,3.2
1,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The Secretary State talks of the lack of fuss ...,The right hon. Gentleman may not have heard fu...,North Tayside.,The Secretary of State's geography is terrible.,Would not the £40 million gap in the assumptio...,0,0,SNP,John Swinney,0.4
2,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,May I assure the right hon. Gentleman that he ...,Will the hon. Gentleman clarify what he meant ...,NaN,NaN,NaN,1,1,Lab,John McAllion,4.5
3,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,"Is the hon. Gentleman aware that, ever since t...","I rise as one who, in the past, has not been a...",It would be inappropriate to allow the debate ...,"Why are the problems of so many of our people,...",Today's debate is the first of this sort under...,0,1,Lab,Ian Davidson,3.1
4,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The hon. Gentleman will notice that the Childr...,I thank the hon. Gentleman for his generosity ...,NaN,NaN,NaN,1,1,Lab,Maria Fyfe,2.3


In [152]:
# count=0
# for i in range(1250):
#     if(data['manual_speech'][i]==data['vote_speech'][i]):
#         count+=1
# print("count=",count)
        

In [153]:
# data['id'].unique()

In [154]:
# data['title'].unique()

In [155]:
# data['manual_motion'].unique()

In [156]:
# data['govt_opp_motion'].unique()

In [157]:
# data['motion_party_affiliation'].unique()

In [158]:
# data['manual_speech'].unique()

In [159]:
# data['vote_speech'].unique()

In [160]:
# data['party_affiliation'].unique()

In [161]:
# data['name'].unique()

In [162]:
# data['rebellion'].unique()

In [163]:
# data['motion_party_affiliation'].unique()

In [4]:
data.describe()

,id,manual_motion,govt_opp_motion,manual_speech,vote_speech,rebellion
count,1251.000000,1251.000000,1251.000000,1251.000000,1251.000000,1251.000000
mean,76.455635,0.396483,0.414868,0.564349,0.569944,1.173141
std,33.891100,0.489362,0.492896,0.496040,0.495282,1.944312
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,0.000000,0.000000,0.000000,0.000000,0.200000
50%,80.000000,0.000000,0.000000,1.000000,1.000000,0.600000
75%,105.000000,1.000000,1.000000,1.000000,1.000000,1.400000
max,129.000000,1.000000,1.000000,1.000000,1.000000,25.100000


In [5]:
new_data=data[['id','manual_motion', 'utt1','utt2','utt3','utt4','utt5','govt_opp_motion','manual_speech','vote_speech','party_affiliation','motion_party_affiliation']]

In [6]:
new_data

,id,manual_motion,utt1,utt2,utt3,utt4,utt5,govt_opp_motion,manual_speech,vote_speech,party_affiliation,motion_party_affiliation
0,1,1,I trust that the House will not feel that ther...,If one considers the argument from a religious...,I accept that we shall all face death at one t...,"When we listen to my hon. Friend's case, we mu...",I do not think that we want that sort of socie...,1,0,0,Lab,Lab
1,2,1,The Secretary State talks of the lack of fuss ...,The right hon. Gentleman may not have heard fu...,North Tayside.,The Secretary of State's geography is terrible.,Would not the £40 million gap in the assumptio...,1,0,0,SNP,Lab
2,2,1,May I assure the right hon. Gentleman that he ...,Will the hon. Gentleman clarify what he meant ...,NaN,NaN,NaN,1,1,1,Lab,Lab
3,2,1,"Is the hon. Gentleman aware that, ever since t...","I rise as one who, in the past, has not been a...",It would be inappropriate to allow the debate ...,"Why are the problems of so many of our people,...",Today's debate is the first of this sort under...,1,0,1,Lab,Lab
4,2,1,The hon. Gentleman will notice that the Childr...,I thank the hon. Gentleman for his generosity ...,NaN,NaN,NaN,1,1,1,Lab,Lab
5,3,0,"Does my hon. Friend agree that, whereas last w...",I follow on from what my hon. Friend the Membe...,"That is correct, but PEPs were not.",There was a penalty for withdrawing from PEPs ...,"My hon. Friend makes a valid point, which I wa...",0,1,1,Con,Con
6,3,0,Does my hon. Friend agree that part of the dis...,NaN,NaN,NaN,NaN,0,1,1,Con,Con
7,3,0,"First, I must declare an interest as the chair...",I cannot understand how the new scheme will be...,I want to highlight some mechanical problems. ...,I am greatly relieved that the Government have...,The change does not meet the Government's orig...,0,1,1,Con,Con
8,4,1,My hon. Friend mentions Staffordshire and the ...,NaN,NaN,NaN,NaN,1,0,1,Lab,Lab
9,4,1,My right hon. Friend probably heard the interv...,NaN,NaN,NaN,NaN,1,0,0,Con,Lab


In [7]:
new_data.describe()

,id,manual_motion,govt_opp_motion,manual_speech,vote_speech
count,1251.000000,1251.000000,1251.000000,1251.000000,1251.000000
mean,76.455635,0.396483,0.414868,0.564349,0.569944
std,33.891100,0.489362,0.492896,0.496040,0.495282
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,0.000000,0.000000,1.000000,1.000000
75%,105.000000,1.000000,1.000000,1.000000,1.000000
max,129.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
new_data["utt2"].fillna(" ", inplace=True)
new_data["utt3"].fillna(" ", inplace=True)
new_data["utt4"].fillna(" ", inplace=True)
new_data["utt5"].fillna(" ", inplace=True)

/home/adityaasinha28/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [9]:
new_data

,id,manual_motion,utt1,utt2,utt3,utt4,utt5,govt_opp_motion,manual_speech,vote_speech,party_affiliation,motion_party_affiliation
0,1,1,I trust that the House will not feel that ther...,If one considers the argument from a religious...,I accept that we shall all face death at one t...,"When we listen to my hon. Friend's case, we mu...",I do not think that we want that sort of socie...,1,0,0,Lab,Lab
1,2,1,The Secretary State talks of the lack of fuss ...,The right hon. Gentleman may not have heard fu...,North Tayside.,The Secretary of State's geography is terrible.,Would not the £40 million gap in the assumptio...,1,0,0,SNP,Lab
2,2,1,May I assure the right hon. Gentleman that he ...,Will the hon. Gentleman clarify what he meant ...,,,,1,1,1,Lab,Lab
3,2,1,"Is the hon. Gentleman aware that, ever since t...","I rise as one who, in the past, has not been a...",It would be inappropriate to allow the debate ...,"Why are the problems of so many of our people,...",Today's debate is the first of this sort under...,1,0,1,Lab,Lab
4,2,1,The hon. Gentleman will notice that the Childr...,I thank the hon. Gentleman for his generosity ...,,,,1,1,1,Lab,Lab
5,3,0,"Does my hon. Friend agree that, whereas last w...",I follow on from what my hon. Friend the Membe...,"That is correct, but PEPs were not.",There was a penalty for withdrawing from PEPs ...,"My hon. Friend makes a valid point, which I wa...",0,1,1,Con,Con
6,3,0,Does my hon. Friend agree that part of the dis...,,,,,0,1,1,Con,Con
7,3,0,"First, I must declare an interest as the chair...",I cannot understand how the new scheme will be...,I want to highlight some mechanical problems. ...,I am greatly relieved that the Government have...,The change does not meet the Government's orig...,0,1,1,Con,Con
8,4,1,My hon. Friend mentions Staffordshire and the ...,,,,,1,0,1,Lab,Lab
9,4,1,My right hon. Friend probably heard the interv...,,,,,1,0,0,Con,Lab


In [10]:
for i in new_data.index:
    #speech = new_data.at[i,'utt1']
    #processed_speech = preprocess_tweet(speech)
    new_data.at[i, 'utt1'] = new_data.at[i, 'utt1'] + " " + new_data.at[i, 'utt2'] +" " + new_data.at[i, 'utt3'] + " " +new_data.at[i, 'utt4'] + " " +new_data.at[i, 'utt5']    
    
   

In [11]:
new_data['utt1'][0]

'I trust that the House will not feel that there is a lack of compassion on any side of the argument about such a dreadful and painful subject. Many of us have had experiences similar to that of my hon. Friend the Member for Bassetlaw, either directly in our own families or on the periphery of decisions to turn off life-sustaining machines. Therefore, no one should be seen as lacking compassion. I find it strange that, having explained the present position in law, my hon. Friend suggested that legislation was required. Even in the case that he cited, the House of Lords has made the position quite clear. If a doctor administers medicine to alleviate pain and suffering and that results in the shortening of a person\'s life—although that does not necessarily happen—it is not euthanasia or murder. Much depends on the intent of the person administering the medicine. If one considers the argument from a religious point of view, different people will have different attitudes. I understand tha

In [12]:
new_data=new_data[['id','manual_motion', 'utt1','govt_opp_motion','manual_speech','vote_speech','party_affiliation','motion_party_affiliation']]

In [13]:
def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    #tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()
    
    #f=open("stopwords.txt", 'r')
    #wordn=f.read()
    #stopwords = nltk.corpus.stopwords.words('english')
    #stopwords += re.split('; |, |\*|\n',wordn)
    
    hansard_stopwords = ({'friend', 'honourable', 'hon', 'gentleman', 'lady','give', 'way', 'house', 'amendment', 'beg', 'move','member', 'question', 'green', 'white', 'paper', 'bill', 'statement', 'government', 'prime', 'minister','opposition', 'party', 'mr', 'rose'})
    sklhansard_stopwords = text.ENGLISH_STOP_WORDS.union(hansard_stopwords)
    

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if word not in sklhansard_stopwords:
               	processed_tweet.append(word)

    return ' '.join(processed_tweet)

In [14]:
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


In [15]:
def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

In [16]:
for i in new_data.index:
    speech = new_data.at[i,'utt1']
    processed_speech = preprocess_tweet(speech)
    new_data.at[i, 'utt1'] = processed_speech

In [17]:
new_data['utt1'].values

array(['trust feel lack compassion argument dreadful painful subject experiences similar bassetlaw directly families periphery decisions turn lifesustaining machines seen lacking compassion strange having explained present position law suggested legislation required case cited lords position quite clear doctor administers medicine alleviate pain suffering results shortening persons does necessarily euthanasia murder depends intent person administering medicine considers argument religious point view different people different attitudes understand practising christian convince agnostic atheist role state govern society outlawed capital punishment said wrong proposed state regulate death industry passing legislation immediately concede motive entirely different legislating people die dangerous thing friends say right die sure accept argument rights normally assume duties duty incumbent right die accept shall face death time right ask people help die people duty help die argue individual 

In [18]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(new_data['utt1'].values)
X = tokenizer.texts_to_sequences(new_data['utt1'].values)
X = pad_sequences(X)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
hansard_stopwords = ({'friend', 'honourable', 'hon', 'gentleman', 'lady','give', 'way', 'house', 'amendment', 'beg', 'move','member', 'question', 'green', 'white', 'paper', 'bill', 'statement', 'government', 'prime', 'minister','opposition', 'party', 'mr', 'rose'})
sklhansard_stopwords = text.ENGLISH_STOP_WORDS.union(hansard_stopwords)
vectorizer = TfidfVectorizer(min_df=5, max_df = 1.0, 
ngram_range=(1,3), sublinear_tf=True, use_idf =True, 
stop_words=sklhansard_stopwords)

In [20]:
val=new_data['utt1']
val

0       trust feel lack compassion argument dreadful p...
1       secretary state talks lack fuss local settleme...
2       assure right probably thinking sad matters lon...
3       aware beginning urban aid programme projects l...
4       notice children scotland act enacted operation...
5       does agree week treated peter pan economics ca...
6       does agree disaster governments plans announce...
7       declare chairman investment management busines...
8       mentions staffordshire birmingham northern rel...
9       right probably heard intervention hayes harlin...
10      does right agree issue just numbers abandonmen...
11      aware gloucestershire police lowest increase c...
12      welcome right aware representations oldham met...
13      right aware public claim ruth coleman liberal ...
14      grateful secretary state giving impressed rema...
15      point local communities supporting council tax...
16      barnsley council year received highest settlem...
17      having

In [21]:
Y=new_data['manual_speech']
Y

0       0
1       0
2       1
3       0
4       1
5       1
6       1
7       1
8       0
9       0
10      0
11      0
12      1
13      0
14      0
15      1
16      1
17      1
18      0
19      0
20      1
21      0
22      1
23      1
24      1
25      1
26      1
27      1
28      0
29      1
       ..
1221    1
1222    1
1223    1
1224    1
1225    0
1226    0
1227    0
1228    1
1229    1
1230    0
1231    0
1232    0
1233    0
1234    1
1235    0
1236    1
1237    1
1238    1
1239    0
1240    0
1241    1
1242    1
1243    1
1244    1
1245    1
1246    0
1247    1
1248    1
1249    1
1250    0
Name: manual_speech, Length: 1251, dtype: int64

In [22]:
#X_train, X_test, Y_train, Y_test = train_test_split(val,Y, test_size = 0.33, random_state = 42)

In [23]:
#X_train.shape

In [24]:

#speeches_train_corpus = vectorizer.fit_transform(X_train.tolist())

#print("speeches_train_corpus",speeches_train_corpus)
#speeches_test_corpus = vectorizer.transform(X_test.tolist())

In [22]:
val_train=vectorizer.fit_transform(val.tolist())
val_train

<1251x3150 sparse matrix of type '<class 'numpy.float64'>'
	with 58917 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
# speeches_train_corpus = transformer.fit_transform(speeches_train_corpus)
# speeches_test_corpus = transformer.transform(speeches_test_corpus)
val_train = transformer.fit_transform(val_train)

In [24]:
val_train.shape

(1251, 3150)

In [25]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
new_data["party_code"] = lb_make.fit_transform(new_data["party_affiliation"])
new_data

lb_mp = LabelEncoder()
new_data["motion_party_code"] = lb_mp.fit_transform(new_data["motion_party_affiliation"])
new_data



/home/adityaasinha28/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/adityaasinha28/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,manual_motion,utt1,govt_opp_motion,manual_speech,vote_speech,party_affiliation,motion_party_affiliation,party_code,motion_party_code
0,1,1,trust feel lack compassion argument dreadful p...,1,0,0,Lab,Lab,8,3
1,2,1,secretary state talks lack fuss local settleme...,1,0,0,SNP,Lab,11,3
2,2,1,assure right probably thinking sad matters lon...,1,1,1,Lab,Lab,8,3
3,2,1,aware beginning urban aid programme projects l...,1,0,1,Lab,Lab,8,3
4,2,1,notice children scotland act enacted operation...,1,1,1,Lab,Lab,8,3
5,3,0,does agree week treated peter pan economics ca...,0,1,1,Con,Con,0,0
6,3,0,does agree disaster governments plans announce...,0,1,1,Con,Con,0,0
7,3,0,declare chairman investment management busines...,0,1,1,Con,Con,0,0
8,4,1,mentions staffordshire birmingham northern rel...,1,0,1,Lab,Lab,8,3
9,4,1,right probably heard intervention hayes harlin...,1,0,0,Con,Lab,0,3


In [26]:
#num_data=new_data[['manual_motion','govt_opp_motion','vote_speech','id']]
num_data=new_data[['govt_opp_motion','id','manual_motion','party_code','motion_party_code']]
num_data

,govt_opp_motion,id,manual_motion,party_code,motion_party_code
0,1,1,1,8,3
1,1,2,1,11,3
2,1,2,1,8,3
3,1,2,1,8,3
4,1,2,1,8,3
5,0,3,0,0,0
6,0,3,0,0,0
7,0,3,0,0,0
8,1,4,1,8,3
9,1,4,1,0,3


In [27]:
from scipy.sparse import hstack as hs
from numpy import hstack

In [28]:
new_train_data=hs([val_train,num_data])

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(new_train_data,Y, test_size = 0.0, random_state = 42)
X_train

<1251x3155 sparse matrix of type '<class 'numpy.float64'>'
	with 63012 stored elements in Compressed Sparse Row format>

In [30]:
def create_GRU(X_train_rec):
    embed_dim = 128
    lstm_out = 196
    max_features=200
#     model = Sequential()
#     model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
#     model.add(SpatialDropout1D(0.4))
#     model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
    
#     kernel_size = 2
#     model = Sequential()
#     model.add(Dense(60, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
#     model.add(Conv1D(600, kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(Conv1D(300, kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(Conv1D(150, kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(Conv1D(75, kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(Flatten())
#     model.add(Dense(600))
#     model.add(Dropout(0.5))
#     model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    from keras.layers import LSTM, Convolution1D, Flatten, Dropout, Dense, GRU

#     max_review_length = 1600
#     embed_dim = 300
#     model = Sequential()
#     model.add(Embedding(200, embed_dim,input_length = X_train.shape[1]))
#     model.add(Convolution1D(64, 3, border_mode='same'))
#     model.add(Convolution1D(32, 3, border_mode='same'))
#     model.add(Convolution1D(16, 3, border_mode='same'))
#     model.add(Flatten())
#     model.add(Dropout(0.2))
#     model.add(Dense(180,activation='sigmoid'))
#     model.add(Dropout(0.2))
#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    model = Sequential()
    ##embedding_size = 8
    #model.add(Embedding(input_dim=num_words,
#                     output_dim=embedding_size,
#                     input_length=max_tokens,
#                     name='layer_embedding'))
    model.add(Embedding(200, embed_dim,input_length = X_train_rec.shape[1]))
    model.add(GRU(units=16, name = "gru_1",return_sequences=True))
    model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
    model.add(GRU(units=4, name= "gru_3"))
    model.add(Dense(1, activation='sigmoid',name="dense_1"))
    #optimizer = Adam(lr=1e-4)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    #create model
#     model = Sequential()
#     model.add(Dense(60, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
#     model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
#     #Compile model
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [31]:
def create_LSTM():
    embed_dim = 128
    lstm_out = 196
    max_features=200
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
    
#     kernel_size = 2
#     model = Sequential()
#     model.add(Dense(60, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
#     model.add(Conv1D(600, kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(Conv1D(300, kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(Conv1D(150, kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(Conv1D(75, kernel_size, padding='valid', activation='relu', strides=1))
#     model.add(Flatten())
#     model.add(Dense(600))
#     model.add(Dropout(0.5))
#     model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    #from keras.layers import LSTM, Convolution1D, Flatten, Dropout, Dense, GRU

#     max_review_length = 1600
#     embed_dim = 300
#     model = Sequential()
#     model.add(Embedding(200, embed_dim,input_length = X_train.shape[1]))
#     model.add(Convolution1D(64, 3, border_mode='same'))
#     model.add(Convolution1D(32, 3, border_mode='same'))
#     model.add(Convolution1D(16, 3, border_mode='same'))
#     model.add(Flatten())
#     model.add(Dropout(0.2))
#     model.add(Dense(180,activation='sigmoid'))
#     model.add(Dropout(0.2))
#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
#     model = Sequential()
#     ##embedding_size = 8
#     #model.add(Embedding(input_dim=num_words,
# #                     output_dim=embedding_size,
# #                     input_length=max_tokens,
# #                     name='layer_embedding'))
#     model.add(Embedding(200, embed_dim,input_length = X_train.shape[1]))
#     model.add(GRU(units=16, name = "gru_1",return_sequences=True))
#     model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
#     model.add(GRU(units=4, name= "gru_3"))
#     model.add(Dense(1, activation='sigmoid',name="dense_1"))
#     #optimizer = Adam(lr=1e-4)
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    #create model
#     model = Sequential()
#     model.add(Dense(60, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
#     model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
#     #Compile model
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [32]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.layers import Dense, Dropout, Activation , GRU
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D, Bidirectional
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM, Convolution1D, Flatten, Dropout, Dense
# evaluate model with standardized dataset

# estimator = KerasClassifier(build_fn=create_baseline, epochs=10, batch_size=5, verbose=0)
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
# results = cross_val_score(estimator,X_train, Y_train, cv=kfold)
# print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# model=create_LSTM()
# model.fit(X_train,Y_train, validation_split=0.30, epochs=15,batch_size=32 )

folds_no = 10
kf = StratifiedKFold(n_splits=folds_no, shuffle=True)

In [33]:
val[4]

'notice children scotland act enacted operation time recollect conservative make available extra money local authorities wrong say money allocated year implementation measures children scotland act accept million sure exact figure said year unlike previous years money available thank generosity giving doing scottish national pretends difference conservative labour parties heard right secretary state make clear things different near future local authority services housing particular pretend labour attitude previous quite dishonest'

In [34]:
num_data_arr=np.array(num_data)
num_data.iloc[[1,2]]

,govt_opp_motion,id,manual_motion,party_code,motion_party_code
1,1,2,1,11,3
2,1,2,1,8,3


In [35]:
#GRU K fold
#new_train_data,Y
cvscores = []
X_sparse=new_train_data.todense()
for train_index, test_index in kf.split(val,Y):
    
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = val[train_index], val[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    num_data_train, num_data_test = num_data.iloc[train_index],num_data.iloc[test_index]
    #num_data_train = num_data.iloc[[train_index]]
    #print(train_index)
    #new_train_k=hs([X_train,num_data_train])
    #new_test_k=hs([X_test,num_data_test])
    X_val_train=vectorizer.fit_transform(X_train.tolist())
    X_val_test=vectorizer.transform(X_test.tolist())
    

    transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
# speeches_train_corpus = transformer.fit_transform(speeches_train_corpus)
# speeches_test_corpus = transformer.transform(speeches_test_corpus)
    X_val_train = transformer.fit_transform(X_val_train)
    X_val_test=transformer.transform(X_val_test)
    #print("y_train=",y_train)
    #print("y_test=",y_test)
    X_new_train_data=hs([X_val_train,num_data_train])
    X_new_test_data=hs([X_val_test,num_data_test])
    #modelGRU=create_GRU(X_new_train_data)
     #embed_dim = 128
    #lstm_out = 196
    #max_features=200
    
    
    embed_dim = 128
    lstm_out = 196
    max_features=200
#     model = Sequential()
#     model.add(Embedding(1000, embed_dim,input_length = (X_new_train_data.toarray()).shape[1]))
#     model.add(SpatialDropout1D(0.4))
#     model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model = Sequential()
    ##embedding_size = 8
    #model.add(Embedding(input_dim=num_words,
#                     output_dim=embedding_size,
#                     input_length=max_tokens,
#                     name='layer_embedding'))
    model.add(Embedding(200, 128,input_length = (X_new_train_data.toarray()).shape[1]))
    model.add(GRU(units=16, name = "gru_1",return_sequences=True))
    model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
    model.add(GRU(units=4, name= "gru_3"))
    model.add(Dense(1, activation='sigmoid',name="dense_1"))
    #optimizer = Adam(lr=1e-4)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    #X_new_train_data
    model.fit(X_new_train_data.toarray(), y_train, epochs=10, batch_size=32, verbose=1)
    # evaluate the model
    scores = model.evaluate(X_new_test_data.toarray(), y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    print("Values are" ,model.metrics_names, scores)
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
#X_new_train_data
#X_new_test_data


Epoch 1/10
1125/1125 [==============================] - 126s 112ms/step - loss: 0.6810 - acc: 0.5529
Epoch 2/10
1125/1125 [==============================] - 121s 108ms/step - loss: 0.6313 - acc: 0.6720
Epoch 3/10
1125/1125 [==============================] - 107s 95ms/step - loss: 0.5339 - acc: 0.7369
Epoch 4/10
1125/1125 [==============================] - 120s 107ms/step - loss: 0.4431 - acc: 0.8089
Epoch 5/10
1125/1125 [==============================] - 97s 87ms/step - loss: 0.3773 - acc: 0.8667
Epoch 6/10
1125/1125 [==============================] - 97s 86ms/step - loss: 0.3518 - acc: 0.8764
Epoch 7/10
1125/1125 [==============================] - 96s 85ms/step - loss: 0.3273 - acc: 0.8764
Epoch 8/10
1125/1125 [==============================] - 94s 84ms/step - loss: 0.3104 - acc: 0.8853
Epoch 9/10
1125/1125 [==============================] - 94s 84ms/step - loss: 0.3061 - acc: 0.8756
Epoch 10/10
126/126 [==============================] - 2s 19ms/step
acc: 85.71%
Values are ['loss', 'a

KeyboardInterrupt: 

In [39]:
#GRU K fold
#new_train_data,Y
cvscores = []
X_sparse=new_train_data.todense()
for train_index, test_index in kf.split(val,Y):
    
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = val[train_index], val[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    num_data_train, num_data_test = num_data.iloc[train_index],num_data.iloc[test_index]
    #num_data_train = num_data.iloc[[train_index]]
    #print(train_index)
    #new_train_k=hs([X_train,num_data_train])
    #new_test_k=hs([X_test,num_data_test])
    X_val_train=vectorizer.fit_transform(X_train.tolist())
    X_val_test=vectorizer.transform(X_test.tolist())
    

    transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
# speeches_train_corpus = transformer.fit_transform(speeches_train_corpus)
# speeches_test_corpus = transformer.transform(speeches_test_corpus)
    X_val_train = transformer.fit_transform(X_val_train)
    X_val_test=transformer.transform(X_val_test)
    #print("y_train=",y_train)
    #print("y_test=",y_test)
    X_new_train_data=hs([X_val_train,num_data_train])
    X_new_test_data=hs([X_val_test,num_data_test])
    #modelGRU=create_GRU(X_new_train_data)
     #embed_dim = 128
    #lstm_out = 196
    #max_features=200
    
    
    embed_dim = 128
    lstm_out = 196
    max_features=200
#     model = Sequential()
#     model.add(Embedding(1000, embed_dim,input_length = (X_new_train_data.toarray()).shape[1]))
#     model.add(SpatialDropout1D(0.4))
#     model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model = Sequential()
    ##embedding_size = 8
    #model.add(Embedding(input_dim=num_words,
#                     output_dim=embedding_size,
#                     input_length=max_tokens,
#                     name='layer_embedding'))
    model.add(Embedding(200, 128,input_length = (X_new_train_data.toarray()).shape[1]))
    model.add(GRU(units=16, name = "gru_1",return_sequences=True))
    model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
    model.add(GRU(units=4, name= "gru_3"))
    model.add(Dense(1, activation='sigmoid',name="dense_1"))
    #optimizer = Adam(lr=1e-4)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    #X_new_train_data
    model.fit(X_new_train_data.toarray(), y_train, epochs=10, batch_size=32, verbose=1)
    # evaluate the model
    scores = model.evaluate(X_new_test_data.toarray(), y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    print("Values are" ,model.metrics_names, scores)
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
#X_new_train_data
#X_new_test_data


Epoch 1/10
1125/1125 [==============================] - 94s 83ms/step - loss: 0.6245 - acc: 0.7200
Epoch 2/10
1125/1125 [==============================] - 92s 82ms/step - loss: 0.4358 - acc: 0.8533
Epoch 3/10
1125/1125 [==============================] - 92s 82ms/step - loss: 0.3308 - acc: 0.9102
Epoch 4/10
1125/1125 [==============================] - 92s 82ms/step - loss: 0.2767 - acc: 0.9209
Epoch 5/10
1125/1125 [==============================] - 92s 82ms/step - loss: 0.2529 - acc: 0.9227
Epoch 6/10
1125/1125 [==============================] - 92s 82ms/step - loss: 0.2394 - acc: 0.9227
Epoch 7/10
1125/1125 [==============================] - 92s 82ms/step - loss: 0.2302 - acc: 0.9218
Epoch 8/10
1125/1125 [==============================] - 93s 82ms/step - loss: 0.2151 - acc: 0.9218
Epoch 9/10
1125/1125 [==============================] - 92s 82ms/step - loss: 0.2035 - acc: 0.9307
Epoch 10/10
126/126 [==============================] - 2s 19ms/step
acc: 93.65%
Values are ['loss', 'acc'] [0

1127/1127 [==============================] - 92s 82ms/step - loss: 0.3098 - acc: 0.9193
Epoch 5/10
1127/1127 [==============================] - 100s 89ms/step - loss: 0.2701 - acc: 0.9264
Epoch 6/10
1127/1127 [==============================] - 101s 90ms/step - loss: 0.2410 - acc: 0.9264
Epoch 7/10
1127/1127 [==============================] - 101s 89ms/step - loss: 0.2260 - acc: 0.9264
Epoch 8/10
1127/1127 [==============================] - 101s 89ms/step - loss: 0.2080 - acc: 0.9237
Epoch 9/10
1127/1127 [==============================] - 101s 89ms/step - loss: 0.1884 - acc: 0.9352
Epoch 10/10
124/124 [==============================] - 3s 25ms/step
acc: 89.52%
Values are ['loss', 'acc'] [0.285524814840286, 0.8951612845543893]
Epoch 1/10
1127/1127 [==============================] - 109s 96ms/step - loss: 0.6501 - acc: 0.6637
Epoch 2/10
1127/1127 [==============================] - 100s 88ms/step - loss: 0.4878 - acc: 0.8296
Epoch 3/10
1127/1127 [==============================] - 93s 82ms/

In [57]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

91.77% (+/- 1.92%)


In [224]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(max_iter=200, alpha=1e-5, hidden_layer_sizes=(100,200,300,400,300,200,100), random_state=1)

In [226]:
MLP.fit(X_train,Y_train)
y_pred=MLP.predict(X_test)

ValueError: Found array with 0 sample(s) (shape=(0, 3156)) while a minimum of 1 is required.

In [92]:
model = create_baseline()

In [98]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred=model.predict(X_test)
y_pred=y_pred.round()

In [106]:
DL_scores = [0,0,0,0]
DL_speech_acc, DL_speech_pres, DL_speech_rec, DL_speech_f1 = 0, 0, 0, 0
DL_speech_acc = DL_speech_acc+accuracy_score(Y_test, y_pred)
DL_speech_pres = DL_speech_pres+precision_score(Y_test, y_pred)
DL_speech_rec = DL_speech_rec+recall_score(Y_test, y_pred)
DL_speech_f1 = DL_speech_f1+f1_score(Y_test, y_pred)

print(DL_speech_acc, DL_speech_pres, DL_speech_rec, DL_speech_f1)

folds_no=10
egs_no=413

DL_temp = [(DL_speech_acc*folds_no)*egs_no, (DL_speech_pres/folds_no)*egs_no, (DL_speech_rec/folds_no)*egs_no, (DL_speech_f1/folds_no)*egs_no]

for x in range(len(DL_scores)):
    DL_scores[x] += DL_temp[x]


DL_scores = [x/2 for x in DL_scores]

# PRINT SPEECH CLASSIFICATION RESULTS #
print('    Accuracy (%)  Precision      Recall         F1')
print('DL', DL_scores[0]/1251, DL_scores[1]/1251, DL_scores[2]/1251, DL_scores[3]/1251)

0.6803874092009685 0.7872340425531915 0.6166666666666667 0.691588785046729
    Accuracy (%)  Precision      Recall         F1
DL 1.123101518784972 0.012994710614487134 0.010179189981348254 0.011415913997773745


In [64]:

def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=speeches_train_corpus.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [66]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=10, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
results = cross_val_score(estimator,speeches_train_corpus, Y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 67.30% (2.52%)


In [277]:
X

array([[  0,   0,   0, ...,  81, 860, 700],
       [  0,   0,   0, ..., 106, 239,  31],
       [  0,   0,   0, ...,  14, 115, 252],
       ...,
       [  0,   0,   0, ..., 641, 130, 876],
       [  0,   0,   0, ..., 560, 959,  52],
       [  0,   0,   0, ..., 839, 895, 919]], dtype=int32)

In [219]:
arr=np.array(X)
arr

array([[  0,   0,   0, ...,  81, 860, 700],
       [  0,   0,   0, ..., 106, 239,  31],
       [  0,   0,   0, ...,  14, 115, 252],
       ...,
       [  0,   0,   0, ..., 641, 130, 876],
       [  0,   0,   0, ..., 560, 959,  52],
       [  0,   0,   0, ..., 839, 895, 919]], dtype=int32)

In [220]:
arr.shape

(1251, 342)

In [221]:
idnew=pd.get_dummies(data['id']).values

In [222]:
idnew

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [223]:
vspeechnew=pd.get_dummies(data['vote_speech']).values

In [224]:
vspeechnew

array([[1, 0],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]], dtype=uint8)

In [225]:
manualmotnew=pd.get_dummies(data['manual_motion']).values

In [226]:
manualmotnew

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [266]:
processed_arr=[]
temp_arr=[]
for i in range(len(arr)):
    #temp_arr.append(arr[i].append(new_data['id']))
    temp_arr.append(idnew[i])
    temp_arr.append(vspeechnew[i])
    #temp_arr.append(new_data['govt_opp_motion'])
    temp_arr.append(manualmotnew[i])
    processed_arr.append(temp_arr)
    temp_arr=[]

In [267]:
#processed_arr=np.array(processed_arr)
processed_arr[0]


[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=uint8), array([1, 0], dtype=uint8), array([0, 1], dtype=uint8)]

In [281]:
new_data
X=new_data[['id','manual_motion','govt_opp_motion','vote_speech']]
X

,id,manual_motion,govt_opp_motion,vote_speech
0,1,1,1,0
1,2,1,1,0
2,2,1,1,1
3,2,1,1,1
4,2,1,1,1
5,3,0,0,1
6,3,0,0,1
7,3,0,0,1
8,4,1,1,1
9,4,1,1,0


In [283]:
Y=new_data['manual_speech']
Y

0       0
1       0
2       1
3       0
4       1
5       1
6       1
7       1
8       0
9       0
10      0
11      0
12      1
13      0
14      0
15      1
16      1
17      1
18      0
19      0
20      1
21      0
22      1
23      1
24      1
25      1
26      1
27      1
28      0
29      1
       ..
1221    1
1222    1
1223    1
1224    1
1225    0
1226    0
1227    0
1228    1
1229    1
1230    0
1231    0
1232    0
1233    0
1234    1
1235    0
1236    1
1237    1
1238    1
1239    0
1240    0
1241    1
1242    1
1243    1
1244    1
1245    1
1246    0
1247    1
1248    1
1249    1
1250    0
Name: manual_speech, Length: 1251, dtype: int64

In [284]:

def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [288]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 92.66% (2.25%)


In [264]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 3, 128)            256000    
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 3, 128)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [265]:
Y = pd.get_dummies(data['manual_speech']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(838, 3) (838, 2)
(413, 3) (413, 2)


In [249]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(100,200,300,1000,300,200,100),verbose = True)
classifier.fit(X_train,Y_train)

ValueError: setting an array element with a sequence.

In [250]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C = 1.0)

In [251]:

clf.fit(X_train, Y_train)
confidence = clf.score(X_test, y_test)
print(confidence)

ValueError: setting an array element with a sequence.

In [164]:
y_pred = classifier.predict(X_test)

In [165]:
accuracy_score(Y_test, y_pred)

0.5181598062953995

In [252]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7


ValueError: setting an array element with a sequence.

In [203]:
validation_size = 300

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.52
acc: 0.56
